# Predict Survival of the Titanic Passengers

Dataset Source: [https://ww2.amstat.org/publications/jse/v3n3/datasets.dawson.html](https://ww2.amstat.org/publications/jse/v3n3/datasets.dawson.html)

## Table of Contents
- [Load Libraries](#load_libraries)
- [Access data](#access_data)
- [Split Data into Training and Test set](#training_test)
- [Build Logistic Regression Model](#build_model)
- [Predict for Test data](#test_data)
- [Evaluate the Model](#evaluate_model)

<a id="load_libraries"></a>
## Load Libraries

The Spark and Python libraries that you need are preinstalled in the notebook environment and only need to be loaded.

Run the following cell to load the libraries you will work with in this notebook:

In [18]:
# PySpark Machine Learning Library
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row, SQLContext

import os
import sys
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from numpy import array

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Library for confusion matrix, precision, test error
from pyspark.mllib.evaluation import MulticlassMetrics
# Library For Area under ROC curve and Area under precision-recall curve
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Assign resources to the application
sqlContext = SQLContext(sc)

In [19]:
# The data will be loaded into an array.
# This is the summary of the data structure, including the column position and name.
# The first filed starts from position 0. 

# 0 ID    -  Identification Code.
# 1 LOW  -  Low birth weight (0: >= 2500, 1: < 2500 g) -  target variable
# 2 Age - Mother's age in years
# 3 Race - Race (1: White, 2: Black, 3: Other)
# 4 Smoke - Smoking status during pregnancy
# 5 PTL - History of premature labor (1: None, 2: One, 3: Two, etc)
# 6 HT - History of hypertension(1: No, 2: Yes)
# 7 UI - Presence of Uterine irritability (1: No, 2: Yes)
# 8 FTV - Number of physician visits during the first trimester (1: None, 2: One, 3: Two, etc)

# Label is a target variable. PersonInfo is a list of independent variables besides unique identifier

LabeledDocument = Row("PersonID", "PersonInfo", "label")

# Define a function that parses the raw CSV file and returns an object of type LabeledDocument

def parseDocument(line):
    values = [str(x) for x in line.split(',')] 
    if (values[1]<'2500'):
      Low = 1.0
    else:
      Low = 0.0
        
    textValue = str(values[1]) + " " + str(values[2])+" " + str(values[3])
    return LabeledDocument(values[5], textValue, Low)

<a id="access_data"></a>
## Access Data
Before you can access data in the data file in the Object Storage, you must setup the Spark configuration with your Object Storage credentials. 

To do this, click on the cell below and select the **Insert to code > Insert Spark Session DataFrame** function from the Files tab below the data file you want to work with.

<div class="alert alert-block alert-info">The following code contains the credentials for a file in your IBM Cloud Object Storage. Delete the code starting from `from pyspark.sql import SparkSession` line before you run the cell.</div>

In [20]:
# Object Storage Credentials
import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'JVTHf52ulbECLgcNi_HtACeV8GS2PPzazxUhftWUrxlS',
    'service_id': 'iam-ServiceId-c3f94a6d-ea3d-4889-aba0-2d5f40560f3c',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_31f75a6160394f40bab252e191667cc8_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')


Now let's load the data into a `Spark RDD` and output the number of rows and first 5 rows.
Each project you create has a bucket in your object storage. You can get the bucket name from the project Settings page. Change the string `BUCKET` to the bucket name

In [21]:
data = sc.textFile(cos.url('lowbwt.csv', 'wesleyclarkassignment3lowbirthwei-donotdelete-pr-rli3ie5o1dyvdn'))
print "Total records in the data set:", data.count()
print "The first 5 rows"
data.take(5)

Total records in the data set: 190
The first 5 rows


[u'ID,LOW,AGE,RACE,SMOKE,PTL,HT,UI,FTV',
 u'85,0,19,2,0,0,0,1,0',
 u'86,0,33,3,0,0,0,0,3',
 u'87,0,20,1,1,0,0,0,1',
 u'88,0,21,1,1,0,0,1,2']

Crate DataFrame from RDD

In [22]:
#Load the data into a dataframe, parse it using the function above
documents = data.filter(lambda s: "ID" not in s).map(parseDocument)
lowbwtData = documents.toDF() # ToDataFrame
print "Number of records: " + str(lowbwtData.count())
print "First 5 records: "
lowbwtData.take(5)

Number of records: 189
First 5 records: 


[Row(PersonID=u'0', PersonInfo=u'0 19 2', label=1.0),
 Row(PersonID=u'0', PersonInfo=u'0 33 3', label=1.0),
 Row(PersonID=u'0', PersonInfo=u'0 20 1', label=1.0),
 Row(PersonID=u'0', PersonInfo=u'0 21 1', label=1.0),
 Row(PersonID=u'0', PersonInfo=u'0 18 1', label=1.0)]

<a id="training_test"></a>
## Split Data into Training and Test set

We divide the data into training and test set.  The training set is used to build the model to be used on future data, and the test set is used to evaluate the model.

In [23]:
# Divide the data into training and test set
(train, test) = lowbwtData.randomSplit([0.8, 0.2])
print "Number of records in the training set: " + str(train.count())
print "Number of records in the test set: " + str(test.count())
# Output first 20 records in the training set
print "First 20 records in the training set: "
train.show()

Number of records in the training set: 162
Number of records in the test set: 27
First 20 records in the training set: 
+--------+----------+-----+
|PersonID|PersonInfo|label|
+--------+----------+-----+
|       0|    0 15 2|  1.0|
|       0|    0 16 1|  1.0|
|       0|    0 16 2|  1.0|
|       0|    0 16 3|  1.0|
|       0|    0 17 2|  1.0|
|       0|    0 17 2|  1.0|
|       0|    0 17 3|  1.0|
|       0|    0 17 3|  1.0|
|       0|    0 17 3|  1.0|
|       0|    0 18 1|  1.0|
|       0|    0 18 1|  1.0|
|       0|    0 18 1|  1.0|
|       0|    0 18 1|  1.0|
|       0|    0 18 2|  1.0|
|       0|    0 19 1|  1.0|
|       0|    0 19 1|  1.0|
|       0|    0 19 2|  1.0|
|       0|    0 19 3|  1.0|
|       0|    0 19 3|  1.0|
|       0|    0 19 3|  1.0|
+--------+----------+-----+
only showing top 20 rows



<a id="build_model"></a>
## Build Logistic Regression Model

We use the Pipeline of SparkML to build the Logistic Regression Model

In [24]:
# set up Logistic Regression using Pipeline of SparkML
tokenizer = Tokenizer(inputCol="PersonInfo", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [25]:
# set up Logistic Regression Model
# the stages are executed in order
model = pipeline.fit(train)
#[stage.coefficients for stage in model.stages if hasattr(stage, "coefficients")]
# model.stages[2].intercept
model.stages[2].coefficients

SparseVector(262144, {})

<a id="test_data"></a>
## Predict for Test data

In [26]:
# Make predictions on test documents and print columns of interest
prediction = model.transform(test)
selected = prediction.select("PersonInfo", "prediction", "probability")
for row in selected.collect():
    print row
#for row in prediction.collect():
#    print row

Row(PersonInfo=u'0 17 1', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 17 3', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 18 1', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 19 1', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 19 3', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 21 1', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 22 1', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 22 1', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 23 1', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 28 3', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 32 3', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 33 1', prediction=1.0, probability=DenseVector([0.0, 1.0]))
Row(PersonInfo=u'0 36 1', prediction=1.0, probabilit

In [27]:
#Tabulate the predicted outcome
prediction.select("prediction").groupBy("prediction").count().show(truncate=False)

+----------+-----+
|prediction|count|
+----------+-----+
|1.0       |27   |
+----------+-----+



In [28]:
#Tabulate the actual outcome
prediction.select("label").groupBy("label").count().show(truncate=False)

+-----+-----+
|label|count|
+-----+-----+
|1.0  |27   |
+-----+-----+



In [29]:
# This table shows:
# 1. The number of survived passengers predicted as survived
# 2. The number of survived passengers predicted as not survived
# 3. The number of not survived passengers predicted as survived
# 4. The number of not survived passengers predicted as not survived

prediction.crosstab('label', 'prediction').show()

+----------------+---+
|label_prediction|1.0|
+----------------+---+
|             1.0| 27|
+----------------+---+



<a id="evaluate_model"></a>
## Evaluate the Model

We evaluate the model on a training set and on a test set.  The purpose is to measure the model's predictive accuracy, including the accuracy for new data.

In [30]:
# Evaluate the Logistic Regression model on a training set
# Select (prediction, true label) and compute test error
pred_lr=model.transform(train).select("prediction", "label")
eval_lr=MulticlassClassificationEvaluator (
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_lr=eval_lr.evaluate(pred_lr)
# create RDD
predictionAndLabels_lr=pred_lr.rdd
metrics_lr=MulticlassMetrics(predictionAndLabels_lr)
precision_lr=metrics_lr.precision(1.0)
recall_lr=metrics_lr.recall(1.0)
f1Measure_lr = metrics_lr.fMeasure(1.0, 1.0)
print("F1 Measure = %s" % f1Measure_lr)
print ("Test Accuracy = %s" %accuracy_lr)
print ("Test Error = %s" % (1-accuracy_lr))
print ("Precision = %s" %precision_lr)
print ("Recall = %s" %recall_lr)

F1 Measure = 1.0
Test Accuracy = 1.0
Test Error = 0.0
Precision = 1.0
Recall = 1.0


In [31]:
# Evaluate the Logistic Regression model on a test set
# Select (prediction, true label) and compute test error
pred_lr=model.transform(test).select("prediction", "label")
eval_lr=MulticlassClassificationEvaluator (
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_lr=eval_lr.evaluate(pred_lr)
# create RDD
predictionAndLabels_lr=pred_lr.rdd
metrics_lr=MulticlassMetrics(predictionAndLabels_lr)
precision_lr=metrics_lr.precision(1.0)
recall_lr=metrics_lr.recall(1.0)
f1Measure_lr = metrics_lr.fMeasure(1.0, 1.0)
print("F1 Measure = %s" % f1Measure_lr)
print ("Test Accuracy = %s" %accuracy_lr)
print ("Test Error = %s" % (1-accuracy_lr))
print ("Precision = %s" %precision_lr)
print ("Recall = %s" %recall_lr)

F1 Measure = 1.0
Test Accuracy = 1.0
Test Error = 0.0
Precision = 1.0
Recall = 1.0


In [32]:
bin_lr=BinaryClassificationMetrics(predictionAndLabels_lr)

# Area under precision-recall curve
print("Area under PR = %s" % bin_lr.areaUnderPR)
# Area under precision-recall curve
print("Area under ROC = %s" % bin_lr.areaUnderROC)

Area under PR = 1.0
Area under ROC = 1.0
